In [1]:
import matplotlib.pyplot as plt;
import numpy as np;
import numpy.random as rand;
import scipy.linalg as sla;
import control.matlab as con;

con.use_numpy_matrix(flag=False, warn=True)

In [2]:
# Rješenje algebarske Riccatijeve jednadžbe (CARE) pomoću Hamiltonove matrice.
# Funkcija vraća stabilizirajuće poz. def. rješenje od A.T*X + X*A + Q - X*B*B.T*X = 0.
def CARE( A, B, Q ):
    n = A.shape[0];
    
    # Formiramo Hamiltonovu matricu.
    # (Ovo možemo i s pozivima hstack/vstack.)
    H = np.block( [[A, B @ B.T], [Q, -A.T]] );
    
    # Odredimo svoj. vrijednosti od H koje su u lijevoj poluravnini.
    [Lam, X] = sla.eig( H );
    perm = np.nonzero( np.real(Lam) < 0 );
    
    # Dakle, Lam[perm] su u lijevoj poluravnini.
    # Uzmimo pripadne svoj. vektore (stupce iz X).
    # Prvih n redaka tih stupaca je X1, zadnjih n je X2.
    X1 = X[0:n, perm[0]];
    X2 = X[n:2*n, perm[0]];
    
    # Funkcija vraća X = -X2 * inv(X1).
    # Umjesto računanja inverza, rješavamo lin. sustave.
    # solve( A, B ) rješava A*X = B, tj. X = inv(A)*B.
    # Dakle, Xt = solve( X1.T, -X2.T ) daje Xt = -inv(X1.T)*X2.T,
    # pa je X = Xt.T = -X2 * inv(X1)
    Xt = sla.solve( X1.T, -X2.T );
    X = Xt.T;
    
    return X;

In [3]:
# Testiramo funkciju CARE.
sys_t = con.rss( 5, 3, 2 );
A = np.array( sys_t.A );
B = np.array( sys_t.B );
C = np.array( sys_t.C );

Q = C.T @ C;

X = CARE( A, B, Q );

# Provjerimo rezidual.
print( 'Riccati rezidual = ', sla.norm( A.T @ X + X @ A + Q - X @ B @ B.T @ X, 'fro' ) );

# U control.matlab postoji funkcija care koja računa ovo isto.
# Funkcija još vraća i feedback matricu F = B.T*X, te svoj. vrijednosti
# sustava zatvorene petlje (tj. svoj. vr. od A - B*F )
[X_control, Lam, F] = con.care( A, B, Q );
print( 'X = X_control? ---> ', sla.norm( X - X_control ) );

Riccati rezidual =  3.3871803102423014e-14
X = X_control? --->  4.843774254763098e-15


In [4]:
# Rješenje LQR problema:
# - za sustav sys_t sa x'(t) = A*x(t) + B*u(t)
# - minimizirati J(u) = \int_0^\infty x(t).T*Q*x(t) + u(t).T*R*u(t) dt, uz Q>=0, R>0.
def LQR( sys_t, Q, R ):
    A = sys_t.A;
    B = sys_t.B;
    
    # Treba riješiti Ric. jednadžbu
    # A.T*X + X*A + Q - X*B*inv(R)*B.T*X = 0.
    # Uoči: care iz control.matlab može primiti R kao dodatni parametar,
    # to jest, možemo odmah dobiti traženo rješenje pomoću
    # [X_control, Lam, F] = care( A, B, Q, R );
    
    # Pozovimo ipak našu funkciju CARE:
    # R12 = R^(1/2)
    R12 = sla.sqrtm( R );
    # BB = B * inv(R12)
    BBt = sla.solve( R12.T, B.T ); BB = BBt.T;

    X = CARE( A, BB, Q );
    
    # Feedback matrica: F = -inv(R)*B.T*X
    F = -B.T @ X;
    F = sla.solve( R, F );
    
    # Svoj. vrijednosti zatvorene petlje.
    Lam = sla.eigvals( A + B @ F );
    
    return [F, X, Lam];

In [5]:
# Testiramo funkciju LQR.

# Uzmimo neku poz. def. matricu, npr.
R = np.array( [ [2, 1], [1, 10]] ); 

[F, X, Lam] = LQR( sys_t, Q, R );

# Sve bi trebale biti u lijevoj poluravnini.
print( 'LQR svoj. vrijednosti: \n', Lam, '\n' );

# U control.matlab postoji analogna funkcija lqr.
[F_control, X_control, Lam_control] = con.lqr( sys_t, Q, R );

# Trebala bi vraćati isto, osim što je kod nas
# F = -inv(R)*B.T * X,
# a u control.matlab je
# F = inv(R)*B.T * X.
print( 'lqr svoj. vrijednosti: \n', Lam_control, '\n' );

print( 'X = X_control?  ---> ', sla.norm( X - X_control, 'fro' ) );
print( 'F = -F_control? ---> ', sla.norm( F + F_control, 'fro' ) );

LQR svoj. vrijednosti: 
 [-1.12900731-2.90741031e+00j -1.12900731+2.90741031e+00j
 -1.69447722+9.37347081e-17j -3.31177547-2.65705577e-31j
 -2.60683721+1.42144369e-17j] 

lqr svoj. vrijednosti: 
 [-1.1290073+2.9074104j -1.1290073-2.9074104j -1.6944772+0.j
 -2.6068373+0.j        -3.3117754+0.j       ] 

X = X_control?  --->  2.1531115580474164e-14
F = -F_control? --->  2.8183111782206252e-15
